In [1]:
from pathlib import Path
import sys, json
from typing import Dict, List, Tuple, Any

import numpy as np
import cv2
import torch
from PIL import Image
from torchvision.transforms.functional import resize
from accelerate import Accelerator

# Label Anything
sys.path.append(str(Path.cwd().parent))
sys.path.append(str(Path.cwd().parent / 'label_anything'))
from label_anything import LabelAnything
from label_anything.data import get_preprocessing, utils
from label_anything.data.transforms import PromptsProcessor

# ==== 定数 ====
ANNOT_DIR = Path.cwd() / "annotations"
IMAGE_DIR = Path.cwd() / "images"
IMAGE_SIZE = 1024
MASK_SIDE = 256
CLASS_NAME_TO_ID = {
    "handrail": 1, "midrail": 2, "toeboard": 3,
    "base_board": 3, "baseboard": 3,  # COCO名の揺れ対応
    "手すり": 1, "中桟": 2, "巾木": 3,
}

# ==== 可視化ユーティリティ（必要なら使用） ====
def draw_masks(img: Image.Image, masks: torch.Tensor, colors):
    masked_image = resize(img.copy(), MASK_SIDE)
    for i, mask in enumerate(masks):
        mask = mask.numpy()
        masked_image = np.where(np.repeat(mask[:, :, np.newaxis], 3, axis=2),
                                np.asarray(colors[i % len(colors)], dtype="uint8"),
                                masked_image)
    masked_image = masked_image.astype(np.uint8)
    return cv2.addWeighted(np.array(resize(img, MASK_SIDE)), 0.3, masked_image, 0.7, 0)

def draw_boxes(img: Image.Image, boxes: torch.Tensor, colors):
    img = np.array(img)
    for i, cat in enumerate(boxes):
        for x1,y1,x2,y2 in cat:
            cv2.rectangle(img, (int(x1),int(y1)), (int(x2),int(y2)), colors[i % len(colors)], 2)
    return img

def draw_points(img: Image.Image, points: torch.Tensor, colors):
    img = np.array(img)
    for i, cat in enumerate(points):
        for x,y in cat:
            cv2.circle(img, (int(x),int(y)), 5, colors[i % len(colors)], -1)
    return img

def draw_all(img: Image.Image, masks, boxes, points, colors):
    segmented_image = draw_masks(img, masks, colors)
    img = Image.fromarray(segmented_image)
    img = resize(img, 1024)
    img = Image.fromarray(draw_boxes(img, boxes, colors))
    img = Image.fromarray(draw_points(img, points, colors))
    return img

def get_image(image_tensor: torch.Tensor) -> Image.Image:
    MEAN = np.array([123.675, 116.280, 103.530]) / 255
    STD  = np.array([58.395,  57.120,  57.375 ]) / 255
    x = image_tensor.numpy()
    x = (x * STD[:, None, None]) + MEAN[:, None, None]
    x = (x * 255).astype(np.uint8)
    return Image.fromarray(np.moveaxis(x, 0, -1))

# ==== JSON ローダ（COCO / 簡易） ====
def _mask_from_polygons(size_hw, polygons):
    H, W = int(size_hw[0]), int(size_hw[1])
    m = np.zeros((H, W), dtype=np.uint8)
    for poly in polygons or []:
        if not poly: continue
        pts = np.array(poly, dtype=np.float32).reshape(-1, 2)
        pts = np.round(pts).astype(np.int32)
        cv2.fillPoly(m, [pts], 255)
    return m

def _mask_from_rle(size_hw, counts):
    from pycocotools import mask as mask_utils  # RLEを使わないなら未インストールでも可
    rle = {"size": [int(size_hw[0]), int(size_hw[1])], "counts": counts}
    m = mask_utils.decode(rle)
    if m.ndim == 3: m = m[..., 0]
    return (m.astype(np.uint8) * 255)

def load_ann_any_json(json_path: Path, expected_stem: str | None = None) -> Dict[str, Dict[int, list]]:
    out = {"bboxes": {}, "points": {}, "masks": {}}
    if not json_path.exists():
        return out

    data = json.load(open(json_path, "r", encoding="utf-8"))
    is_coco = all(k in data for k in ("images","categories","annotations"))

    if not is_coco:
        # 簡易形式: {'bboxes':{'1':[[x1,y1,x2,y2],...]}, 'points':{'2':[[x,y],...]}, 'masks':{'3':[entry,...]}}
        for k in ("bboxes","points"):
            raw = data.get(k, {})
            if isinstance(raw, dict):
                for cid, arr in raw.items():
                    try: cid = int(cid)
                    except: continue
                    if isinstance(arr, list):
                        out[k].setdefault(cid, []).extend(arr)
        raw_masks = data.get("masks", {})
        if isinstance(raw_masks, dict):
            for cid, entries in raw_masks.items():
                try: cid = int(cid)
                except: continue
                for entry in (entries or []):
                    fmt = str(entry.get("format","")).lower()
                    size = entry.get("size", None)
                    if not size: continue
                    if fmt == "polygons":
                        m = _mask_from_polygons(size, entry.get("polygons", []))
                        if (m>0).any(): out["masks"].setdefault(cid, []).append(m)
                    elif fmt == "rle":
                        m = _mask_from_rle(size, entry.get("counts"))
                        if (m>0).any(): out["masks"].setdefault(cid, []).append(m)
        return out

    # COCO形式
    images = data.get("images", [])
    anns   = data.get("annotations", [])

    # stem一致 → 1枚のみならフォールバック → 部分一致
    img_entry = None
    if expected_stem is not None:
        for im in images:
            if Path(str(im.get("file_name",""))).stem == expected_stem:
                img_entry = im; break
    if img_entry is None and len(images)==1:
        img_entry = images[0]
    if img_entry is None and expected_stem is not None:
        for im in images:
            if expected_stem in Path(str(im.get("file_name",""))).stem:
                img_entry = im; break
    if img_entry is None:
        return out

    image_id = img_entry["id"]
    H, W = int(img_entry.get("height", 0)), int(img_entry.get("width", 0))
    tanns = [a for a in anns if a.get("image_id")==image_id]

    def add(dic, cid, v): dic.setdefault(int(cid), []).append(v)

    for a in tanns:
        cid = int(a.get("category_id"))

        # bbox [x,y,w,h] → [x1,y1,x2,y2]
        bb = a.get("bbox")
        if isinstance(bb, (list,tuple)) and len(bb)==4:
            x,y,w,h = bb
            x1,y1,x2,y2 = float(x), float(y), float(x)+float(w), float(y)+float(h)
            if x2>x1 and y2>y1:
                add(out["bboxes"], cid, [x1,y1,x2,y2])

        # keypoints → points（v>0のみ）
        kps = a.get("keypoints")
        if isinstance(kps, list) and len(kps)>=3:
            for i in range(0,len(kps),3):
                xk, yk, v = kps[i], kps[i+1], kps[i+2]
                if v and xk is not None and yk is not None:
                    add(out["points"], cid, [float(xk), float(yk)])

        # segmentation → masks（RLE / polygons, 空配列は無視）
        seg = a.get("segmentation")
        if seg is not None:
            try:
                if isinstance(seg, dict) and "counts" in seg and "size" in seg:
                    m = _mask_from_rle(seg["size"], seg["counts"])
                    if (m>0).any(): add(out["masks"], cid, m)
                elif isinstance(seg, list) and seg and isinstance(seg[0], dict) and "counts" in seg[0]:
                    for r in seg:
                        m = _mask_from_rle(r["size"], r["counts"])
                        if (m>0).any(): add(out["masks"], cid, m)
                elif isinstance(seg, list) and len(seg)>0:
                    m = _mask_from_polygons([H,W], seg)
                    if (m>0).any(): add(out["masks"], cid, m)
            except Exception:
                # pycocotools未インストールでRLEが来た場合などは無視（polygonsを推奨）
                pass

    # 全ゼロマスクの最終除外（保険）
    for cid, arrs in list(out["masks"].items()):
        out["masks"][cid] = [m for m in arrs if isinstance(m, np.ndarray) and (m>0).any()]

    return out

def union_class_ids(dicts_per_support: List[Dict[str, Dict[int, list]]]) -> List[int]:
    s = set()
    for d in dicts_per_support:
        for k in ("bboxes","points","masks"):
            s |= set(d.get(k, {}).keys())
    return sorted(s)

# ==== 小技：空マスク除外 & 重心点補完 ====
def filter_empty_masks(masks_per_img: Dict[int, List[np.ndarray]]):
    for cid, arrs in list(masks_per_img.items()):
        masks_per_img[cid] = [m for m in arrs if isinstance(m, np.ndarray) and m.ndim==2 and (m>0).any()]

def mask_centroid(mask: np.ndarray):
    ys, xs = np.nonzero(mask > 0)
    if len(xs)==0: return None
    return int(xs.mean()), int(ys.mean())


/Users/rinotsuka/code/papers/LabelAnything2/LabelAnything/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ==== モデル/前処理 ====
accelerator = Accelerator(cpu=True)
device = accelerator.device

la = LabelAnything.from_pretrained("pasqualedem/label_anything_sam_1024_coco")

# ====== ここから変更（クエリ/サポートを別フォルダに）======
QUERY_DIR   = Path.cwd() / "query_images"    # ←新規
SUPPORT_DIR = Path.cwd() / "support_images"  # ←新規

def open_rgb(p: Path) -> Image.Image:
    return Image.open(p).convert("RGB")

# クエリ画像（1枚想定）
query_paths = sorted(
    list(QUERY_DIR.glob("*.jpg")) + list(QUERY_DIR.glob("*.jpeg")) +
    list(QUERY_DIR.glob("*.png")) + list(QUERY_DIR.glob("*.JPG")) +
    list(QUERY_DIR.glob("*.PNG"))
)
assert len(query_paths) >= 1, "クエリ画像は query_images に1枚だけ置いてください。"
query_orig = open_rgb(query_paths[0])

# サポート画像（1枚以上）
support_paths = sorted(
    list(SUPPORT_DIR.glob("*.jpg")) + list(SUPPORT_DIR.glob("*.jpeg")) +
    list(SUPPORT_DIR.glob("*.png")) + list(SUPPORT_DIR.glob("*.JPG")) +
    list(SUPPORT_DIR.glob("*.PNG"))
)
assert len(support_paths) >= 1, "サポート画像を support_images に1枚以上置いてください。"
support_orig_images = [open_rgb(p) for p in support_paths]

# 前処理
preprocess = get_preprocessing({"common": {"custom_preprocess": True, "image_size": IMAGE_SIZE}})
query_image   = preprocess(query_orig)
support_images = [preprocess(img) for img in support_orig_images]

# 元サイズ（(W,H)）
support_sizes: List[Tuple[int,int]] = [img.size for img in support_orig_images]
all_sizes: List[Tuple[int,int]] = [query_orig.size] + support_sizes

prompts_processor = PromptsProcessor(
    long_side_length=IMAGE_SIZE, masks_side_length=MASK_SIDE, custom_preprocess=True
)
# ====== ここまで変更 ======


In [3]:

# ==== JSON → bboxes/points/masks ====  ← ここを置き換え

from pathlib import Path
import json
import unicodedata

def _is_coco_json(path: Path) -> bool:
    try:
        d = json.load(open(path, "r", encoding="utf-8"))
        return isinstance(d, dict) and all(k in d for k in ("images", "annotations", "categories"))
    except Exception:
        return False

# 1) ANNOT_DIR 内から COCO 形式の JSON を1つ見つける（明示指定があるならそれでもOK）
# 例: annotations/dataset.json を使いたいなら coco_json = ANNOT_DIR / "dataset.json"
coco_json = None
for cand in sorted(ANNOT_DIR.glob("*.json")):
    if _is_coco_json(cand):
        coco_json = cand
        break
assert coco_json is not None, "COCO 形式の注釈JSONが annotations/ に見つかりませんでした。"

# 2) サポート画像ごとに、同じ COCO JSON を渡しつつ expected_stem で画像を特定
support_annots: List[Dict[str, Dict[int, list]]] = []
for p in support_paths:
    # 正規化（NFD/NFC 差対策）
    expected = unicodedata.normalize("NFC", p.stem)
    support_annots.append(load_ann_any_json(coco_json, expected_stem=expected))


# # ==== JSON → bboxes/points/masks ====
# support_annots: List[Dict[str, Dict[int, list]]] = []
# for p in support_paths:  # ← フォルダ分離したならこっち！
#     ann_path = (ANNOT_DIR / f"{p.stem}.json").resolve()
#     support_annots.append(load_ann_any_json(ann_path, expected_stem=p.stem))
    

cat_ids = union_class_ids(support_annots) or [1,2,3]
cat_ids = sorted(cat_ids)

bboxes_list: List[Dict[int, List[List[float]]]] = []
points_list: List[Dict[int, List[List[float]]]] = []
masks_list : List[Dict[int, List[np.ndarray]]] = []

for ann in support_annots:
    per_b = {cid: list(ann.get("bboxes", {}).get(cid, [])) for cid in cat_ids}
    per_p = {cid: list(ann.get("points", {}).get(cid, [])) for cid in cat_ids}
    per_m = {cid: list(ann.get("masks",  {}).get(cid, [])) for cid in cat_ids}

    # 小技1: 空マスク除外
    filter_empty_masks(per_m)

    # 小技2: マスクがあるのにポイントが無いクラスに重心点を1つ補完
    # for cid, arrs in per_m.items():
    #     if len(arrs)>0 and len(per_p.get(cid, []))==0:
    #         c = mask_centroid(arrs[0])
    #         if c is not None:
    #             per_p.setdefault(cid, []).append(list(c))

    bboxes_list.append(per_b)
    points_list.append(per_p)
    masks_list.append(per_m)

# bbox を LA 形式に変換
converted_bboxes: List[Dict[int, List[List[float]]]] = []
for img_bboxes, orig_img in zip(bboxes_list, support_orig_images):
    out = {}
    for cid, cat_bboxes in img_bboxes.items():
        out[cid] = [prompts_processor.convert_bbox(b, *orig_img.size, noise=False) for b in cat_bboxes]
    converted_bboxes.append(out)

# 背景 -1
bboxes_list_bg = [{**{-1: []}, **bb} for bb in converted_bboxes]
points_list_bg = [{**{-1: []}, **pp} for pp in points_list]
masks_list_bg  = [{**{-1: []}, **mm} for mm in masks_list]
cat_ids_bg = [-1] + cat_ids

# numpy 化（空でも配列化）
for i in range(len(bboxes_list_bg)):
    for cid in cat_ids_bg:
        bboxes_list_bg[i][cid] = np.array(bboxes_list_bg[i][cid], dtype=np.float32)
        points_list_bg[i][cid] = np.array(points_list_bg[i][cid], dtype=np.float32)
        # masks は ndarray のリストのまま（LA側で 256 に整形）

# tensor 化
bboxes, flag_bboxes = utils.annotations_to_tensor(prompts_processor, bboxes_list_bg, support_sizes, utils.PromptType.BBOX)
points, flag_points = utils.annotations_to_tensor(prompts_processor, points_list_bg, support_sizes, utils.PromptType.POINT)
masks,  flag_masks  = utils.annotations_to_tensor(prompts_processor, masks_list_bg,  support_sizes, utils.PromptType.MASK)

flag_examples = utils.flags_merge(flag_bboxes=flag_bboxes, flag_points=flag_points, flag_masks=flag_masks)

# ==== 推論 ====
input_dict = {
    utils.BatchKeys.IMAGES: torch.stack([query_image] + support_images).unsqueeze(0),
    utils.BatchKeys.PROMPT_BBOXES: bboxes.unsqueeze(0),
    utils.BatchKeys.FLAG_BBOXES:   flag_bboxes.unsqueeze(0),
    utils.BatchKeys.PROMPT_POINTS: points.unsqueeze(0),
    utils.BatchKeys.FLAG_POINTS:   flag_points.unsqueeze(0),
    utils.BatchKeys.PROMPT_MASKS:  masks.unsqueeze(0),
    utils.BatchKeys.FLAG_MASKS:    flag_masks.unsqueeze(0),
    utils.BatchKeys.FLAG_EXAMPLES: flag_examples.unsqueeze(0),
    utils.BatchKeys.DIMS: torch.tensor([all_sizes], dtype=torch.int32),
#     utils.BatchKeys.DIMS: torch.tensor([[
#     (query_orig.height, query_orig.width),
#     *[(img.height, img.width) for img in support_orig_images]
# ]], dtype=torch.int32),
}
def dict_to_device(d, device):
    if isinstance(d, torch.Tensor): return d.to(device)
    if isinstance(d, dict): return {k: dict_to_device(v, device) for k,v in d.items()}
    if isinstance(d, list): return [dict_to_device(v, device) for v in d]
    return d
input_dict = dict_to_device(input_dict, device)

with torch.no_grad():
    output = la(input_dict)
logits = output["logits"]
predictions = torch.argmax(logits, dim=1)

# ==== 簡易ログ ====
print("shapes:",
      "bboxes", tuple(bboxes.shape),
      "points", tuple(points.shape),
      "masks",  tuple(masks.shape))
print("flags:",
      "bbox:",  int(flag_bboxes.sum().item()),
      "point:", int(flag_points.sum().item()),
      "mask:",  int(flag_masks.sum().item()),
      "examples:", int(flag_examples.sum().item()))

# ==== 可視化（任意。colorsは任意の配列） ====
colors = [
    (255,255,0),(255,0,0),(0,255,0),(0,0,255),
    (255,0,255),(0,255,255),(255,165,0)
]
drawn_images = [
    draw_all(get_image(img_t), img_masks, img_bboxes, img_points, colors)
    for img_t, img_masks, img_bboxes, img_points in zip(
        support_images, masks, bboxes, points
    )
]
# Image.fromarray(drawn_images[0]).save("debug_support0_overlay.png")



shapes: bboxes (5, 4, 1, 4) points (5, 4, 0, 2) masks (5, 4, 256, 256)
flags: bbox: 15 point: 0 mask: 15 examples: 20


In [4]:
# ===== Save predictions to ./output =====
out_dir = Path("output")
out_dir.mkdir(exist_ok=True)

# predictions: (B, H, W) を想定（このスクリプトでは B=1）
pred = predictions[0].detach().cpu().numpy().astype(np.int32)  # H x W

# query の元画像（予測と同サイズのはず。万が一違えばリサイズ）
qh, qw = query_orig.size[1], query_orig.size[0]  # (H,W)
if pred.shape != (qh, qw):
    pred = cv2.resize(pred, (qw, qh), interpolation=cv2.INTER_NEAREST)

# インデックス → 色 のルックアップ（背景含めて len(cat_ids)+1）
# idx=0: 背景(-1), idx=1..C: cat_ids の順
cat_ids_with_bg = [-1] + cat_ids
# 好きなパレットに変更可
palette = np.array([
    (0,0,0),       # 背景: 黒
    (255,255,0),   # class 1
    (255,0,0),     # class 2
    (0,255,0),     # class 3
    (0,0,255),
    (255,0,255),
    (0,255,255),
    (255,165,0),
    (255,192,203),
], dtype=np.uint8)

num_needed = pred.max() + 1
if num_needed > len(palette):
    # パレット拡張（足りない分は循環）
    extra = np.vstack([palette[1:]] * ((num_needed // (len(palette)-1)) + 1))
    palette = np.vstack([palette[:1], extra[:num_needed-1]])

# カラーマップ画像（H,W,3）
color_map = palette[np.clip(pred, 0, len(palette)-1)]
# 保存（クラス色だけ）
Image.fromarray(color_map).save(out_dir / "query_pred_colormap.png")

# オーバーレイ（元画像と半透明合成）
query_rgb = np.array(query_orig)
overlay = cv2.addWeighted(query_rgb, 0.5, color_map, 0.5, 0.0)
Image.fromarray(overlay).save(out_dir / "query_pred_overlay.png")

# クラス別の2値マスク（背景はスキップ）
for idx, cid in enumerate(cat_ids_with_bg):
    if cid == -1:  # 背景
        continue
    mask_bin = (pred == idx).astype(np.uint8) * 255
    if mask_bin.any():  # そのクラスが1ピクセルでも存在する時だけ保存
        Image.fromarray(mask_bin).save(out_dir / f"class_{cid}_mask.png")

print(f"[saved] {out_dir}/query_pred_colormap.png")
print(f"[saved] {out_dir}/query_pred_overlay.png")


[saved] output/query_pred_colormap.png
[saved] output/query_pred_overlay.png


In [39]:
import time, torch
la.eval().to(device)

# 入力テンソルが意図通りのサイズかざっと確認
print("images:", input_dict[utils.BatchKeys.IMAGES].shape)      # 期待: (1, 1+S, 3, H, W)
print("logits_size target:", (IMAGE_SIZE, IMAGE_SIZE), "or upsampled to original")

t0 = time.time()
with torch.no_grad():
    out = la(input_dict)
t1 = time.time()

print("forward_sec:", round(t1 - t0, 2))
print("logits.shape:", out["logits"].shape)


images: torch.Size([1, 6, 3, 1024, 1024])
logits_size target: (1024, 1024) or upsampled to original
forward_sec: 168.54
logits.shape: torch.Size([1, 4, 1528, 1120])


In [42]:
from label_anything import LabelAnything
la = LabelAnything.from_pretrained("pasqualedem/label_anything_sam_1024_coco")

la.eval()


LabelAnything(
  (model): Lam(
    (image_encoder): ImageEncoderViT(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (blocks): ModuleList(
        (0-11): 12 x Block(
          (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): MLPBlock(
            (lin1): Linear(in_features=768, out_features=3072, bias=True)
            (lin2): Linear(in_features=3072, out_features=768, bias=True)
            (act): GELU(approximate='none')
            (drop): Identity()
          )
        )
      )
      (neck): Sequential(
        (0): Conv2d(768, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): LayerNorm2d()
        (2): Conv2d(2

# チェックポイント確認

In [43]:
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file

repo = "pasqualedem/label_anything_sam_1024_coco"

# safetensors のファイルパスを取得
ckpt = hf_hub_download(repo_id=repo, filename="model.safetensors")

# state_dict をロード
sd = load_file(ckpt)

# キー一覧を確認
keys = list(sd.keys())
print("num tensors:", len(keys))
print(keys[:50])  # 最初の50個だけ表示



num tensors: 417
['model.image_encoder.blocks.0.attn.proj.bias', 'model.image_encoder.blocks.0.attn.proj.weight', 'model.image_encoder.blocks.0.attn.qkv.bias', 'model.image_encoder.blocks.0.attn.qkv.weight', 'model.image_encoder.blocks.0.attn.rel_pos_h', 'model.image_encoder.blocks.0.attn.rel_pos_w', 'model.image_encoder.blocks.0.mlp.lin1.bias', 'model.image_encoder.blocks.0.mlp.lin1.weight', 'model.image_encoder.blocks.0.mlp.lin2.bias', 'model.image_encoder.blocks.0.mlp.lin2.weight', 'model.image_encoder.blocks.0.norm1.bias', 'model.image_encoder.blocks.0.norm1.weight', 'model.image_encoder.blocks.0.norm2.bias', 'model.image_encoder.blocks.0.norm2.weight', 'model.image_encoder.blocks.1.attn.proj.bias', 'model.image_encoder.blocks.1.attn.proj.weight', 'model.image_encoder.blocks.1.attn.qkv.bias', 'model.image_encoder.blocks.1.attn.qkv.weight', 'model.image_encoder.blocks.1.attn.rel_pos_h', 'model.image_encoder.blocks.1.attn.rel_pos_w', 'model.image_encoder.blocks.1.mlp.lin1.bias', 'mod

In [44]:
any("prompt_encoder" in k for k in keys), any("mask_decoder" in k for k in keys), any("neck" in k for k in keys)


(True, True, True)

In [ ]:
# 例：ざっくり件数を見る

enc   = [k for k in keys if "image_encoder"  in k]
prompt= [k for k in keys if "prompt_encoder" in k]
dec   = [k for k in keys if "mask_decoder"   in k]
neck  = [k for k in keys if "neck"           in k]
print(len(enc), "encoder,", len(prompt), "prompt,", len(dec), "decoder,", len(neck), "neck")


177 encoder, 130 prompt, 104 decoder, 12 neck


おお、きれいに出ましたね！数字がすべて物語っています。

---

## 集計結果の意味

* **177 encoder**
  → ViT-B/1024 (SAMベースの image\_encoder) の全ブロック＋LayerNorm＋QKV など。
  → これは **SAM の事前学習済み部分**。

* **130 prompt**
  → `model.prompt_encoder.*` の層。BBox / Point / Mask などをエンコードしてクエリ画像と融合する部分。
  → これは **COCO-20i episodic training で実際に学習された部分**。

* **104 decoder**
  → `model.mask_decoder.*` の層。TwoWayTransformer や FFN を通してクラスごとのマスクを生成。
  → ここも **COCO-20i 学習で更新されている部分**。

* **12 neck**
  → 768 (ViT出力) → 512 (LA内部表現) に写像する射影Conv＋LayerNorm。
  → SAM の出力次元と LabelAnything の内部次元を合わせる“橋渡し”。

---

## 結論

* `label_anything_sam_1024_coco` のチェックポイントには
  ✅ **エンコーダ（SAM ViT-B/1024）**
  ✅ **プロンプトエンコーダ**
  ✅ **マスクデコーダ**
  ✅ **Neck**
  が **すべて含まれている**。
* ただし **学習で更新されたのは「プロンプト＋デコーダ＋Neck」側**で、
  **エンコーダは凍結されて SAM の重みをそのまま持っている**。

---

👉 つまり、「二つある？」の答えは **Yes、含まれている**。
そして「どっちが学習された？」の答えは **デコーダ側（prompt＋mask）だけ**、です。

---

このあと確認したいのは

* 「自分で fine-tune する時に encoder を解凍して学習できるのか」なのか
* 「研究的に、この checkpoint をどう表現すべきか（例：SAM encoder frozen, decoder trained on COCO-20i）」なのか

どちらに近いですか？


いい質問です！
state\_dict の「パラメータ数」や「キー数」が **自然かどうか** を見ていきましょう。

---

## 1. Encoder (177 tensors)

* ViT-B/16 (SAM のエンコーダ) の場合、

  * 12層の Transformer block × (attention, mlp, norm の複数パラメータ)
  * patch embedding / positional encoding / 最終の norm
* これらを合わせると **だいたい 150〜200 個のキー** になるのは普通です。
  → **177** は妥当。

---

## 2. Prompt Encoder (130 tensors)

* LabelAnything では「BBox / Point / Mask を埋め込みに変換するモジュール」。
* Conv / Linear / LayerNorm / Transformer 層を含む。
* この規模感で **100前後のキー**があって自然。
  → **130** という数は想定通り。

---

## 3. Mask Decoder (104 tensors)

* TwoWayTransformer（Depth=2, heads=8, hidden\_dim=512）
* さらに FFN, LayerNorm, Conv upsampling などを含む。
* これで **100個前後のパラメータキー**になるのは自然。
  → **104** は妥当。

---

## 4. Neck (12 tensors)

* Conv (1x1) → LayerNorm → Conv (3x3) → LayerNorm
* これで 2 conv × 2 norm = **4レイヤー分の重み＋バイアスなど**が含まれる。
* 1レイヤーごとに weight と bias のキーができるので **10前後**のキー数は自然。
  → **12** はピッタリ。

---

## 5. まとめ

* **177 (encoder) + 130 (prompt) + 104 (decoder) + 12 (neck) = 423**
* あなたのログは **417 tensors** でした → 若干差があるのは

  * bias が無い Linear / Conv
  * LayerNorm の affine 無効化
    などの設定の違いによるもの。
    でも桁感は完全に自然。

---

✅ 結論：
**これらの数値は自然であり、「エンコーダ＋プロンプト＋デコーダ＋Neck が全部含まれている」ことを裏付けるもの。**

---

👉 確認ですが、知りたいのは

* **「値の意味＝この数字がどんな層に対応しているか」** をもっと掘り下げたいのか、
* それとも **「この数が妥当かどうか」** の確認だけで十分ですか？
